<a href="https://colab.research.google.com/github/angelaxli/Naive-Bayes-S-MARs-Detection/blob/main/REAL_naive_bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
BiocManager::install("Biostrings")
if (!require(caret)) install.packages("caret")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.rstudio.com

Bioconductor version 3.20 (BiocManager 1.30.25), R 4.4.2 (2024-10-31)

Installing package(s) 'BiocVersion', 'Biostrings'

also installing the dependencies ‘zlibbioc’, ‘UCSC.utils’, ‘GenomeInfoDbData’, ‘BiocGenerics’, ‘S4Vectors’, ‘IRanges’, ‘XVector’, ‘GenomeInfoDb’


Loading required package: caret

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘caret’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘listenv’, ‘parallelly’, ‘future’, ‘globals’, ‘shape’, ‘future.apply’, ‘numDeriv’, ‘progressr’, ‘SQUAREM’, ‘diagram’, ‘lava’, ‘prodlim’, ‘proxy’, ‘iterators’, ‘clock’

In [ ]:
library(Biostrings)
library(e1071)
library(tidyverse)
library(caret)

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, saveRDS, setdiff,
    table, tapply, union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors

Loading required package: stats4


Attaching package: ‘S4Vectors’


The following object is masked from ‘package:utils’:

    findMatches


The following objects are masked from ‘package:base’:

    expand.grid, I, unname


Loading required package: IRanges

Loading required package: XVector

Loading required package: GenomeInfoDb


Attac

In [ ]:
# Function to compute k-mer counts for a sequence
compute_kmer_counts <- function(sequence, k = 4) {
  # Convert sequence to a DNAString object
  dna_seq <- DNAString(sequence)

  # Generate all possible k-mers for given k
  kmers <- oligonucleotideFrequency(dna_seq, width = k)

  # Convert named vector to data frame
  as.data.frame(as.list(kmers))
}


In [ ]:
# Apply k-mer counting to each sequence
data <- read.csv("naive bayes data.csv")
data$Sequence <- gsub("[^ATGC]", "", data$Sequence)
k <- 4  # Set the desired k-mer length
kmer_features <- do.call(rbind, lapply(data$Sequence, compute_kmer_counts, k = k))

# Combine k-mer features with the target column
data_prepared <- cbind(kmer_features, Target = data$Target)

In [ ]:
# Split the data into training and test sets
set.seed(123)
train_index <- sample(1:nrow(data_prepared), 0.7 * nrow(data_prepared))
train_data <- data_prepared[train_index, ]
test_data <- data_prepared[-train_index, ]

# Train the Naive Bayes model
model <- naiveBayes(Target ~ ., data = train_data)

# Make predictions on the test set
predictions <- predict(model, test_data)

In [ ]:
#Issue: unbalanced dataset(between class imbalance)
confusion_matrix <- table(Predicted = predictions, Actual = test_data$Target)
print(confusion_matrix)

         Actual
Predicted   0   1
        0  11   0
        1   1 131


In [ ]:
# Evaluate model accuracy
accuracy <- mean(predictions == test_data$Target)
cat("Model Accuracy:", accuracy, "\n")

Model Accuracy: 0.993007 


In [ ]:
test_data$Target <- factor(test_data$Target)
predictions <- factor(predictions, levels = levels(test_data$Target))

#Generate the confusion matrix
confusion <- confusionMatrix(predictions, test_data$Target)

# Display confusion matrix and metrics
print(confusion)

Confusion Matrix and Statistics

          Reference
Prediction   0   1
         0  11   0
         1   1 131
                                          
               Accuracy : 0.993           
                 95% CI : (0.9617, 0.9998)
    No Information Rate : 0.9161          
    P-Value [Acc > NIR] : 5.081e-05       
                                          
                  Kappa : 0.9527          
                                          
 Mcnemar's Test P-Value : 1               
                                          
            Sensitivity : 0.91667         
            Specificity : 1.00000         
         Pos Pred Value : 1.00000         
         Neg Pred Value : 0.99242         
             Prevalence : 0.08392         
         Detection Rate : 0.07692         
   Detection Prevalence : 0.07692         
      Balanced Accuracy : 0.95833         
                                          
       'Positive' Class : 0               
                              

In [ ]:
# Extract specific metrics
accuracy <- confusion$overall['Accuracy']
precision <- confusion$byClass['Pos Pred Value']  # This is Precision
recall <- confusion$byClass['Sensitivity']        # This is Recall
f1_score <- (2 * precision * recall) / (precision + recall)  # F1 Score

cat("Accuracy:", accuracy, "\n")
cat("Precision:", precision, "\n")
cat("Recall:", recall, "\n")
cat("F1 Score:", f1_score, "\n")

Accuracy: 0.993007 
Precision: 1 
Recall: 0.9166667 
F1 Score: 0.9565217 
